In [1]:
#设置每行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
#下载数据集
from sklearn import datasets
iris=datasets.load_iris()
list(iris.keys())
import numpy as np



In [2]:
np.random.seed(2042)

In [3]:
def logprint(key,msg=None):
    type_key = type(key)
    if type_key ==str:
        shape = len(key)
    elif type_key ==np.ndarray:
        shape =key.shape
    else:
        shape =None
    if shape !=None:
        print(("logprint\tmsg:"+str(msg)+"\ttype:"+str(type(key))+"\tshape:"+str(shape)))
    else:
        print(("logprint\tmsg:"+str(msg)+"\ttype:"+str(type(key))))
    print(key)

In [4]:
import numpy as np
#sample 样例数量
x= iris["data"][:,(2,3)]
y=iris["target"]
x_with_bias = np.c_[np.ones([len(x), 1]), x]

test_ratio = 0.2
validation_ratio = 0.2
total_size = len(x_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

x_train = x_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
x_valid = x_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
x_test = x_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]
x_train.shape,x_valid.shape,x_test.shape
y_train.shape,y_valid.shape,y_test.shape
n_classes = y.max()+1

#np.arange 形成一个等差的数组 
#np.[[k_a,k_b,k_c],[v_a,v_b,v_c]] 相当于对矩阵的某些行和列幅值
def to_one_hot(y):
#     n_classes = y.max() + 1
    m = len(y)
    y_one_hot = np.zeros((m, n_classes))
    y_one_hot[np.arange(m), y] = 1
    return y_one_hot
y_train_one_hot = to_one_hot(y_train)
y_valid_one_hot = to_one_hot(y_valid)
y_test_one_hot = to_one_hot(y_test) 
y_train_one_hot.shape
y_valid_one_hot.shape
y_test_one_hot.shape

((90, 3), (30, 3), (30, 3))

((90,), (30,), (30,))

(90, 3)

(30, 3)

(30, 3)

下面实现Softmax function.

回忆一下它的定义：

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [5]:
# 其实就是用代码实现这个数学公式
#p[[p11,p12,...,p1k],...,[pm1,pm2,...,pmk]]
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [6]:
n_features = x_train.shape[1] # == 3 (2 features plus the bias term)
np.random.seed(2042)
theta = np.random.randn(n_features,n_classes)
logprint(theta,"theta初始随机值")

logprint	msg:theta初始随机值	type:<class 'numpy.ndarray'>	shape:(3, 3)
[[ 0.5593025   1.29143558  2.27356226]
 [ 0.68283041 -0.25129965  0.62717215]
 [-0.18749445  0.71044056  1.04302869]]


In [7]:
n_iterations =100001
m_train = len(x_train)
eta = 0.01
epsilon = 1e-7

n_classes = len(np.unique(y_train)) 
alpha=0.1
best_loss =np.infty
for iteration in range(n_iterations):
    logits = x_train.dot(theta)
    y_proba = softmax(logits)
#     y_train_one_hot.shape
    # k*d+1
    gradients_t =np.zeros(theta.shape).T
    xentropy_loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
    l2_loss =  1/2 * np.sum(np.square(theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
#实现一 按照公式推导做法 
#     error =y_proba-y_train_one_hot
#     for k  in range(n_classes):
#         #将theta_k 的梯度填充进去
#         gradients_t[k] = 1./m_train *error.T[k].dot(x_train)
#     gradients = gradients_t.T + np.r_[np.zeros([1, n_classes]), alpha * theta[1:]]
#     theta = theta -eta * gradients    
##实现二
    error =y_proba-y_train_one_hot
    gradients_t=1/m_train *error.T.dot(x_train)
    gradients = gradients_t.T +  np.r_[np.zeros([1, n_classes]), alpha * theta[1:]]
    theta = theta -eta * gradients
#实现三 
#     error =y_proba-y_train_one_hot
#     gradients = 1/m_train * x_train.T.dot(error) + np.r_[np.zeros([1, n_classes]), alpha * theta[1:]]
#     theta = theta - eta * gradients
##加入早停
    logits_v = x_valid.dot(theta)
    y_proba_v = softmax(logits_v)
    xentropy_loss_v = -np.mean(np.sum(y_valid_one_hot * np.log(y_proba_v + epsilon), axis=1))
    l2_loss_v = 1/2 * np.sum(np.square(theta[1:]))
    loss = xentropy_loss_v + alpha * l2_loss_v

    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break        
# logprint(theta,"theta迭代%s"%str(n_iterations))

0 2.561571331332843
500 0.9708231098717285
1000 0.808076973013991
1500 0.7193544146997736
2000 0.6674970337024547
2500 0.6349793995124956
3000 0.6132793603963999
3500 0.598007690058714
4000 0.5867693362133265
4500 0.5781874033461191
5000 0.5714325137264352
5500 0.5659835808488032
6000 0.5615003825992032
6500 0.5577528165943716
7000 0.5545801418248097
7500 0.5518666951805085
8000 0.5495269730074995
8500 0.5474962035408695
9000 0.5457242312651907
9500 0.5441714516840684
10000 0.542806046449744
10500 0.5416020618465286
11000 0.5405380458041201
11500 0.5395960621678977
12000 0.5387609645838005
12500 0.5380198522423288
13000 0.5373616551965559
13500 0.5367768135167515
14000 0.5362570254705775
14500 0.5357950472435338
15000 0.5353845316978185
15500 0.5350198971052539
16000 0.5346962191929354
16500 0.5344091415418948
17000 0.5341548005992403
17500 0.5339297624498602
18000 0.533730969144296
18500 0.5335556928626771
19000 0.5334014965577909
19500 0.5332661999961692
20000 0.5331478503278173
2050

In [8]:
logits = x_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = (np.mean(y_predict == y_valid))
accuracy_score

1.0

In [9]:
#最终测试集准确度 
logits = x_test.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9333333333333333

In [10]:
# 公式写错
# n_iterations =50
# m_train = len(x_train)
# eta = 0.01
# epsilon = 1e-7
# # n_outputs = len(np.unique(y_train)) 
# theta = np.random.rand(n_features,n_classes)
# logprint(theta,"theta初始随机值")
# for iteration in range(n_iterations):
#     logits = x_train.dot(theta)
#     y_proba = softmax(logits)
# #     y_train_one_hot.shape
#     # k*d+1
#     gradients_t =np.zeros(theta.shape).T
#     loss = -np.mean(np.sum(y_train_one_hot * np.log(y_proba + epsilon), axis=1))
#     if iteration % 500 == 0:
#         print((iteration, loss))
# #     print("gradients.shape:\t"+str(gradients.shape))
#     for k  in range(n_classes):
#         np_sum =np.zeros([1,n_features])
#         for i in range(m_train):
#             #第I个实例
#             np_sum +=(y_proba[i][k]-y_train_one_hot[i][k])*x_train[i]  
        
#         #将theta_k 的梯度填充进去
#         gradients_t[k]=np_sum*1/m_train
#     gradients = gradients_t.T
#     theta = theta -eta * gradients
# #     logprint(theta,"theta迭代%s"%str(iteration+1))
# logprint(theta,"theta迭代%s"%str(n_iterations))